In [26]:
#pip install pandas

In [27]:
import json
import pandas as pd
import os
from collections import defaultdict

In [42]:
batter_summary = defaultdict(list)
bowler_summary = defaultdict(list)
match_summary = defaultdict(list)

In [29]:
match_id = defaultdict(list)

In [30]:
# for file_name in os.listdir('odis_json_cricsheet'):
#     if file_name.endswith('.json'):
#         with open(os.path.join('odis_json_cricsheet', file_name), 'r') as file:
#             match_data = json.load(file)
#             match_id[file_name]= match_data['info']['match_type_number']

In [31]:
#match_id.values()

In [32]:
# sum=0
# for i in man_of_match:
#     if '' == match_event[i]:
#         sum+=1
# print(sum)

In [33]:
#with open('C:/Users/fazal/Desktop/Fazal/projects/odis_json_cricsheet/64814.json', 'r') as file:
   # match_data = json.load(file)

In [43]:
def process_file(file_path):
    with open(file_path, 'r') as file:
        match_data = json.load(file)

        match_date = match_data['info']['dates'][0]
        match_venue = match_data['info']['venue']
        match_event = match_data['info'].get('event', {}).get('name',None)
        man_of_match = match_data['info'].get('player_of_match', [None])[0]
        match_winner = match_data['info'].get('outcome', {}).get('winner', None)            
        match_id = match_data['info']['match_type_number']

        if match_id not in match_summary:
            match_summary[match_id] = {
                'match_date': match_date,
                'match_venue' : match_venue,
                'match_event' : match_event,
                'man_of_match' : man_of_match,
                'match_winner' : match_winner,
            }
            
        runs_batter = defaultdict(int)
        balls_batter = defaultdict(int)
        runs_bowler = defaultdict(int)
        balls_bowler = defaultdict(int)
        fours_batter = defaultdict(int)
        sixes_batter = defaultdict(int)
        fours_bowler = defaultdict(int)
        sixes_bowler = defaultdict(int)
        maiden_bowler = defaultdict(int)
        out_notout = defaultdict(int)
        wickets_bowler = defaultdict(int)
        
        for inn in match_data['innings']:
            for over_itr in inn['overs']:
                dot_ball = 0
                for ball in over_itr['deliveries']:
    
                    batter = ball['batter']
    
                    #runs, fours, sixes.....
                    
                    if batter not in runs_batter:
                        runs_batter[batter] = 0
                        fours_batter[batter] = 0
                        sixes_batter[batter] = 0
                    runs_batter[batter] += ball['runs']['batter']
                    if ball['runs']['batter'] == 4:
                        fours_batter[batter] += 1
                    elif ball['runs']['batter']==6:
                        sixes_batter[batter] += 1
    
                    #balls......
                    
                    if batter not in balls_batter:
                        balls_batter[batter] = 0
                    if 'extras' in ball:
                        if 'wides' in ball['extras']:
                            pass
                        else:
                            balls_batter[batter] += 1
                    else:
                        balls_batter[batter] += 1
    
                    #out........
                    
                    if batter not in out_notout:
                        out_notout[batter] = 0
                    if 'wickets' in ball:
                        if ball['wickets'][0]['player_out'] == batter:
                            out_notout[batter] = 1
    
                    ##BOWLER STATS
                    bowler = ball['bowler']
    
                    #runs, fours, sixes.....
                    
                    if bowler not in runs_bowler:
                        runs_bowler[bowler] = 0
                        fours_bowler[bowler] = 0
                        sixes_bowler[bowler] = 0
                    runs_bowler[bowler] += ball['runs']['batter']
                    
                    if 'extras' in ball:
                        if 'wides' in ball['extras']:
                            runs_bowler[bowler] +=ball['extras']['wides']
                        elif 'noballs' in ball['extras']:
                            runs_bowler[bowler] += ball['extras']['noballs']
                    
                    if ball['runs']['batter'] == 4:
                        fours_bowler[bowler] += 1
                    elif ball['runs']['batter']==6:
                        sixes_bowler[bowler] += 1
    
                    #balls......
                    
                    if bowler not in balls_bowler:
                        balls_bowler[bowler] = 0
                    if 'extras' in ball:
                        if ('wides' in ball['extras']) or ('noballs' in ball['extras']):
                            pass
                        else:
                            balls_bowler[bowler] += 1
                    else:
                        balls_bowler[bowler] += 1
    
                    #maiden
                    if 'extras' in ball and ('wides' in ball['extras'] or 'noballs' in ball['extras']):
                        dot_ball = -100
            
                    if ball['runs']['batter']==0:
                        dot_ball += 1
                        
                    #wickets
    
                    if bowler not in wickets_bowler:
                        wickets_bowler[bowler] = 0
                    if 'wickets' in ball:
                        if ball['wickets'][0]['kind'] != 'run out':
                            wickets_bowler[bowler] += 1

                #updating maiden over after end of each over if no runs scored(batter or wides,noballs)            
                if bowler not in maiden_bowler:
                    maiden_bowler[bowler] = 0
                if dot_ball==6:
                    maiden_bowler[bowler] += 1

        for team in match_data['info']['players']:
            for player in match_data['info']['players'][team]:
                if player not in batter_summary:
                    batter_summary[player] = []
                batter_summary[player].append({
                    'match_id' : match_id,
                    'team' : team,
                    'm_o_m' : 1 if player==man_of_match else 0,
                    'runs' : runs_batter[player],
                    'balls' : balls_batter[player],
                    'fours' : fours_batter[player],
                    'sixes' : sixes_batter[player],
                    'out_notout' : out_notout[player],
                })

                if player not in bowler_summary:
                    bowler_summary[player] = []
                bowler_summary[player].append({
                    'match_id' : match_id,
                    'team' : team,
                    'm_o_m' : 1 if player==man_of_match else 0,
                    'runs' : runs_bowler[player],
                    'balls' : balls_bowler[player],
                    'wickets' : wickets_bowler[player],
                    'maiden' : maiden_bowler[player],
                
                })

In [44]:
for file_name in os.listdir('odis_json_cricsheet'):
    if file_name.endswith('.json'):
        process_file(os.path.join('odis_json_cricsheet', file_name))
        

In [ ]:
#batter_summary

In [37]:
# for itr in batter_summary['Shubman Gill']:
#     print(itr)

In [38]:
# bowler_summary['DR Tuffey']

In [46]:
import pandas as pd

# Convert match_summary dictionary to DataFrame
fixed_match_summary = {}

for match_id, match_data in match_summary.items():
    # Ensure match_data is a dictionary (not an empty list)
    if isinstance(match_data, list):
        if len(match_data) > 0:  # Check if list is not empty
            match_data = match_data[0]  # Extract first dictionary
        else:
            continue  # Skip this match if it's an empty list

    fixed_match_summary[match_id] = {
        'match_date': match_data['match_date'][0] if isinstance(match_data['match_date'], list) else match_data['match_date'],
        'match_venue': match_data.get('match_venue', ''),  # Use get() to prevent KeyErrors
        'match_event': match_data.get('match_event', ''),
        'man_of_match': match_data['man_of_match'][0] if isinstance(match_data['man_of_match'], list) else match_data['man_of_match'],
        'match_winner': match_data.get('match_winner', '')
    }

df_match_summary = pd.DataFrame.from_dict(fixed_match_summary, orient='index').reset_index()
df_match_summary.rename(columns={'index': 'match_id'}, inplace=True)

# Convert batter_summary to DataFrame
batter_records = []
for batter in batter_summary.keys():
    for match in batter_summary[batter]:
        match['batter'] = batter  # Add batter name to record
        batter_records.append(match)

df_batter_summary = pd.DataFrame(batter_records)

# Convert bowler_summary to DataFrame
bowler_records = []
for bowler in bowler_summary.keys():
    for match in bowler_summary[bowler]:
        match['bowler'] = bowler  # Add bowler name to record
        bowler_records.append(match)

df_bowler_summary = pd.DataFrame(bowler_records)

# Save DataFrame to CSV
df_batter_summary.to_csv("batter_summary.csv", index=False)
df_bowler_summary.to_csv("bowler_summary.csv", index=False)

print("✅ Batter and Bowler summaries saved successfully!")
df_match_summary.to_csv("match_summary.csv", index=False)

print("✅ Match summary CSV saved successfully!")


✅ Batter and Bowler summaries saved successfully!
✅ Match summary CSV saved successfully!


In [40]:
# df_batter_summary[df_batter_summary['batter']=='PJ Cummins']